In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def get_sentiment_counts(data):
    # Initialize counts
    if type(data) is str:
        data=eval(data)

    positive_count = 0
    negative_count = 0
    neutral_count = 0

    # Iterate through the dictionary values
    for sentiment, score in data.values():
        if sentiment == 'Positive':
            positive_count += 1
        elif sentiment == 'Negative':
            negative_count += 1
        elif sentiment == 'Neutral':
            neutral_count += 1

    # Create a 3-length tuple
    sentiment_counts = [positive_count,neutral_count, negative_count]
    
    return sentiment_counts

In [2]:
from bert_score_eval import get_bertscore_fit,get_bertscore_summary
#sc1,sc2=get_bertscore_fit(df_result)
#print(sc1,sc2)

In [4]:
import pandas as pd

def calculate_summary_metric():
    # Initialize an empty dataframe for the summary
    #summary_df = pd.DataFrame(columns=['Conversation', 'Agent','BertScoreCoefficient','FirstSentiment','LastSentiment'])
    summary_df=[]
    for i in range(0,12):
        # Read the dataframe
        df = pd.read_csv('KData/eval_metric_100_' + str(i) + '.csv')
        BertScoreCoefficient1,BertScoreCoefficient2=get_bertscore_fit(df)
        meanBS1,meanBS2,meanP1,meanP2=get_bertscore_summary(df)
        FirstSentiment1=get_sentiment_counts(df['TopSentimentScores'][1])
        #print(FirstSentiment1)
        FirstSentiment2=get_sentiment_counts(df['TopSentimentScores'][2])
        LastSentiment1=get_sentiment_counts(df['TopSentimentScores'][len(df)-2])
        LastSentiment2=get_sentiment_counts(df['TopSentimentScores'][len(df)-1])
        agent1_data={'Conversation':'eval_metric_100_' + str(i), 'Agent':'Agent1','F1BertScoreCoefficient':BertScoreCoefficient1,'MeanF1BertScore':meanBS1,'MeanPerpexlity':meanP1,'FirstConvSentiment':FirstSentiment1,'LastConvSentiment':LastSentiment1}
        agent2_data={'Conversation':'eval_metric_100_' + str(i), 'Agent':'Agent2','F1BertScoreCoefficient':BertScoreCoefficient2,'MeanF1BertScore':meanBS2,'MeanPerpexlity':meanP2,'FirstConvSentiment':FirstSentiment2,'LastConvSentiment':LastSentiment2}
        summary_df.append(agent1_data)
        summary_df.append(agent2_data)

    
    summary_df = pd.DataFrame(summary_df)

    return summary_df


In [7]:
sum_df=calculate_summary_metric()
sum_df.to_csv('KData/eval_summary_100.csv')

In [8]:
sum_df.head(50)


,Conversation,Agent,F1BertScoreCoefficient,MeanF1BertScore,FirstConvSentiment,LastConvSentiment
0,eval_metric_100_0,Agent1,-0.000291,0.623046,"[0, 0, 5]","[0, 1, 4]"
1,eval_metric_100_0,Agent2,0.000302,0.623046,"[4, 1, 0]","[4, 1, 0]"
2,eval_metric_100_1,Agent1,0.000325,0.583462,"[0, 0, 5]","[3, 0, 2]"
3,eval_metric_100_1,Agent2,0.000255,0.583462,"[5, 0, 0]","[0, 0, 5]"
4,eval_metric_100_2,Agent1,-0.000996,0.576481,"[0, 0, 5]","[5, 0, 0]"
5,eval_metric_100_2,Agent2,-0.001067,0.576481,"[5, 0, 0]","[0, 0, 5]"
6,eval_metric_100_3,Agent1,-0.002205,0.566326,"[3, 0, 2]","[3, 1, 1]"
7,eval_metric_100_3,Agent2,-0.001616,0.566326,"[2, 0, 3]","[4, 1, 0]"
8,eval_metric_100_4,Agent1,-0.000676,0.565159,"[0, 0, 5]","[0, 4, 1]"
9,eval_metric_100_4,Agent2,-0.000556,0.565159,"[3, 2, 0]","[2, 3, 0]"


In [ ]:
from agent_heatmap import create_sentiment_heatmap
create_sentiment_heatmap(df_result,"Agent1")
create_sentiment_heatmap(df_result,"Agent2")